# Import libraries

Import GCMPy library with alias gp

In [1]:
import GCMPy as gp
import math
import random
import matplotlib.pyplot as plt
#%matplotlib inline
import numpy as np
import pandas as pd
from pandas import DataFrame
from scipy.optimize import minimize
import seaborn as sns
sns.set(style='ticks', context='paper')
colors=["#e3c934","#68c4bf","#c51000","#287271"]
sns.set_palette(colors)

## Import Peterson & Barney 1952 as pb52, convert to Bark

In [2]:
pb52=pd.read_csv('pb52.csv')
pbbark = gp.HzToBark(pb52,["F0","F1","F2","F3"])

## Set some parameters

In [3]:
# set dimesnsions m as keys, 
    ## set weight of each dimension w_m as values
dimsvals={'z0':1,'z1':2.953,'z2':.924,'z3':3.420}
dimslist=list(dimsvals.keys())

# set categories to be considered as items in a list
catslist=['vowel']
cat="vowel"

# Define error function

In [4]:
def errorfunc_noc(x,testset,cloud,dimslist,catslist,c):
    #x = [z1,z2,z3]
    # dimslist = [z0, z1, z2, z3]
    # dimsdict = {z0 =1, z1 = z1guess, z2 = z2guess, z3=z3guess}
    # dimslist[0] = z0        dl[1] = z1,       dl[2] = z2.      dl[3] = z3
    #            1            x[0]= z1,          x[1] = z2,      x[2] =z3 
    dimsdict={dimslist[0]:1, dimslist[1]:x[0], dimslist[2]:x[1], dimslist[3]:x[2]}
    choices=gp.multicat(testset,cloud,catslist,dims=dimsdict,c=c)
    accuracy=gp.checkaccuracy(choices,catslist)
    err = accuracy['vowel'+'Acc'].value_counts(normalize=True)['n']
    return err

## Optimize parameters multiple times using 

This takes about 5 million years to do. The lines to pay attention to are
- fitted parameters = c value, w_z1, w_z2, w_z3
- fun = the error rate
- nfev = the number of fits that were tried

In [5]:
pbsmall = gp.gettestset(pbbark,'vowel',3)

In [6]:
test=pbsmall                 #stimuli to categorize
cloud=pbsmall                #cloud to categorize against
dimslist=['z0','z1','z2','z3']  #dimensions to categorize along
catslist=['vowel']           #categories to categorize
c=25                         # c value

name = 'test'                # name of output spreadsheet
n = 3                        # number of loops
#-------------------------------------------

resultslist=[['start','fit','error','evals']]
for i in range(0,n):
    x=np.divide(random.sample(range(0,300),3),100)
        # Get a random sample of 3 numbers between 0 and 300,
           # divide by 100 to get floats between 0 and 3  
    xguess = x
    result = minimize(errorfunc_noc,
                  xguess,  # the initial guess array
                  args=(test,cloud,dimslist,catslist,c),
                  method='Powell',  
                  tol=0.01,  # a 'tolerance' value, smaller means more function evaluation
                 )

    print(i, x)
    print("fitted parameters = ", result.x)
    print("number of function evaluations = ", result.nfev)
    print("error = ", result.fun)
    start = x
    fit = np.round(result.x,3)
    error = result.fun
    evals = result.nfev
    row = [start,fit,error,evals]
    resultslist.append(row)
results=pd.DataFrame(resultslist)
results.columns = results.iloc[0]
results=results[1:]
results.to_csv(name+".csv")    

0 [1.79 0.89 2.21]
fitted parameters =  [3.79       1.27196603 0.973932  ]
number of function evaluations =  30
error =  0.16666666666666666
1 [1.61 1.03 2.32]
fitted parameters =  [3.99196603 1.93983006 1.84786407]
number of function evaluations =  49
error =  0.16666666666666666
2 [1.61 1.78 0.49]
fitted parameters =  [4.61       0.95775858 0.9332171 ]
number of function evaluations =  59
error =  0.13333333333333333
